In [ ]:
%pip install seaborn
%pip install pandas
%pip install scipy
%pip install -U scikit-learn
%pip install pycaret

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as nmp
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sb
from pycaret.classification import *
from pycaret.datasets import get_data

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.6.0-cp310-cp310-win_amd64.whl (11.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Goncalo\\AppData\\Roaming\\Python\\Python310\\site-packages\\~-learn\\.libs\\msvcp140.dll'
Check the permissions.


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import Datasets
launches = pd.read_csv('Launches.csv')
configs = pd.read_csv('Configs.csv')
families = pd.read_csv('Families.csv')
companies = pd.read_csv('Companies.csv')
locations = pd.read_csv('Locations.csv')
missions = pd.read_csv('Missions.csv')

As our dataset is split up between 6 csv files, the dataframe must combine the data from them. For that, we will use joins and, for them to work, the data types must be correctly aligned.

In [ ]:
configs['Family Id'] = configs['Family Id'].astype(int)

rockets = pd.merge(configs, families, how = 'inner', on = 'Family Id')

launch_data = pd.merge(launches, locations, how = 'inner', left_on = 'Location', right_on = 'Orig_Addr')
launch_data = pd.merge(launch_data, rockets, how = 'inner', right_on = 'Config', left_on = 'Rocket Name')
launch_data = pd.merge(launch_data, companies, how = 'inner', left_on = 'Rocket Organisation', right_on = 'Company Name')
launch_data = pd.merge(launch_data, missions, how = 'inner', on = 'Launch Id')

Let's now consider the dataset info and all feature present after merging all csvs.

In [ ]:
launch_data.describe()
launch_data.head()
launch_data.columns

Columns such as the launch id can be ignored, as they are not useful for our analysis. Duplicate rows should also be removed, as well as rows where there is lack of information.

In [ ]:
launch_data = launch_data.drop_duplicates()
launch_data = launch_data.dropna()
launch_data = launch_data.drop(["Launch Id"], axis="columns")
launch_data = launch_data.drop(["Family Id"], axis="columns")
launch_data = launch_data.drop(["No_x"], axis="columns")
launch_data = launch_data.drop(["No_y"], axis="columns")
launch_data = launch_data.drop(["Dum"], axis="columns")
launch_data = launch_data.drop(["Launch Year Mon"], axis="columns")
launch_data = launch_data.drop(["Launch Suborbital"], axis="columns")

converting bad values

In [ ]:
launch_data['Price'] = launch_data['Price'].str.replace('$', '').str.replace(' million', '').str.replace(',', '').astype(float)

# Convert thrust to numeric, removing 'kN'
launch_data['Liftoff Thrust'] = launch_data['Liftoff Thrust'].str.replace(' kN', '').str.replace(',', '').astype(float)

# Convert payload capacities to numeric, removing 'kg'
launch_data['Payload to LEO'] = launch_data['Payload to LEO'].str.replace(' kg', '').str.replace(',', '').astype(float)
launch_data['Payload to GTO'] = launch_data['Payload to GTO'].str.replace(' kg', '').str.replace(',', '').astype(float)

# Convert dimensions to numeric, removing 'm'
launch_data['Rocket Height'] = launch_data['Rocket Height'].str.replace(' m', '').astype(float)
launch_data['Fairing Diameter'] = launch_data['Fairing Diameter'].str.replace(' m', '').astype(float)
launch_data['Fairing Height'] = launch_data['Fairing Height'].str.replace(' m', '').astype(float)

# Clean success rate in families
launch_data['Success Rate'] = launch_data['Success Rate'].str.rstrip('%').astype(float) / 100

As the following plot shows, we have a very imbalanced dataset when considering this specific fields.

In [ ]:
# Count the number of successful and unsuccessful launches
status_counts = launch_data['Launch Status'].value_counts()

# Plot the histogram
plt.figure(figsize=(8, 6))
status_counts.plot(kind='bar', color=['skyblue'])
plt.title('Count of Successful and Unsuccessful Launches', fontsize=16)
plt.xlabel('Launch Status', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
launch_data['Launch Status'].value_counts()

In order to lower its impact, there are a few approaches that can be taken into consiration. In this instance, we decided to go for under-sampling.

In [ ]:

df_majority = launch_data[launch_data['Launch Status'] == 'Success']
df_minority = launch_data[launch_data['Launch Status'] != 'Success']

sample_size = int(len(df_majority)/16)
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=sample_size,     # match number in minority class
                                   random_state=42)  # reproducible results

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_minority, df_majority_downsampled])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the new class distribution
balanced_status = df_balanced['Launch Status'].value_counts()

Let's run a visualization of the balanced dataset, in order to see what happens when we have a balanced dataset.

In [ ]:
# Plot the histogram
plt.figure(figsize=(8, 6))
balanced_status.plot(kind='bar', color=['skyblue'])
plt.title('Count of Successful and Unsuccessful Launches on Balanced Dataframe', fontsize=16)
plt.xlabel('Launch Status', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

In [ ]:
df_balanced['Launch Status'].value_counts()

### 1st Question: does the price influence the success of a Launch?

Let's follow an Hypothesis Testing.

Hypothesis
1) Null Hypothesis (Ho): Price has no influence on the success of a launch.
2) Alternative Hypothesis (Ha): Price influences positively the success of a launch. (Is there a correlation?)

Let's first run a boxplot to see the distribution of the data, check if there are any outliers and if their presence is significant.

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Launch Status', y='Rocket Price', data=df_balanced, width=0.6)
plt.title('Rocket Price by Launch Status')
plt.xlabel('Launch Status')
plt.ylabel('Rocket Price')
plt.show()

The boxplot visualization suggests that rocket price is not a decisive factor in determining launch success, as failures happen across all price ranges.

To examine the relationship between rocket price and launch success, we conducted a point-biserial correlation test. This test is appropriate for analyzing the association between a continuous variable (rocket price) and a dichotomous variable (launch success: 0 or 1). You can know more about this specific test in the following link: https://www.statisticshowto.com/point-biserial-correlation/ 

In [ ]:
rocket_prices = df_balanced['Rocket Price']
launch_success = (df_balanced['Launch Status'] == 'Success').astype(int)
df_balanced['Launch Success Binary'] = df_balanced['Launch Status'].apply(lambda x: 1 if x.lower() == 'success' else 0)

# Calculate point-biserial correlation
correlation, p_value = stats.pointbiserialr(launch_success, rocket_prices)

print(f"Correlation coefficient: {correlation:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant relationship between rocket price and launch success.")
else:
    print("Fail to reject the null hypothesis: There is no significant relationship between rocket price and launch success.")

The results show:

- Correlation coefficient: -0.0107
- P-value: 0.8834

With a p-value of 0.8834, which is much larger than the conventional significance level of 0.05, we fail to reject the null hypothesis. This suggests that there is no statistically significant relationship between rocket price and launch success.

The correlation coefficient of -0.0107 indicates a very weak negative relationship between price and success. However, given the high p-value, this relationship is not statistically significant and is likely due to random chance rather than a true association in the population.

In conclusion, based on this point-biserial correlation test, we fail to reject the null hypothesis that there is no statistically significant relationship between rocket price and launch success. This suggests that there is no evidence to support the claim that rocket prices are directly related to launch success.

---

### 2nd Question: Is there any corporation more likely to succeed in a launch??

Let's follow a Hypothesis Testing, just as in the first question.

Hypothesis
1) Null Hypothesis (Ho): A rocket's corporation has no influence on the success of a launch.
2) Alternative Hypothesis (Ha): The corporation that fabricated the rocket influences positively the success of a launch. (Is there a correlation?)

We'll start with a simple groupby operation on the Launches data frame. We will group the data by the Rocket Organisation, and then count the number of Total launches for each organization, sum the number of Successful Launches, and provide the average number (Success Rate) of successful launches.

In [ ]:
# Group the data by the 'Rocket Organisation' column and calculate the number of launches, successful launches, and success rate.
success_rate = (
    df_balanced.groupby('Rocket Organisation')['Launch Success Binary']
    .agg(['count', 'sum', 'mean'])
    .rename(columns={'count': 'Total Launches', 'sum': 'Successful Launches', 'mean': 'Success Rate'})
)

# Limit the data to only include organizations with at least 5 successful launches, in order to have a more meaningful analysis.
success_rate = success_rate[success_rate['Total Launches'] >= 5]

success_rate = success_rate.sort_values(by=['Success Rate'], ascending=[False])
success_rate.head()

Let's run a visualization to understand which is the best contender to be the top rocket organization 🚀

In [ ]:
plt.figure(figsize=(8, 6))
plt.barh(success_rate.index, success_rate['Success Rate'], color='skyblue')
plt.gca().invert_yaxis() 
plt.title('Top Rocket Organisations by Success Rate', fontsize=16)
plt.xlabel('Success Rate', fontsize=12)
plt.ylabel('Rocket Organisation', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

The visualisation concludes that **SpaceX** is the most successful Rocket Organisation within this dataset, if we consider only the average of Successful Launches per Rocket Organisation.

Let's dive deeper to understand the data, and see if we can find some relationship between the Rocket's Organization and the launch success.

To examine the relationship between corporation and launch success, we conducted a chi-square independence test (you can get more detailed information on this matter here: https://www.jmp.com/en_be/statistics-knowledge-portal/chi-square-test/chi-square-test-of-independence.html). This test evaluates the independence of a feature (launch success: 0 or 1) from another (corporation). 

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_balanced['Rocket Organisation'], df_balanced['Launch Success Binary'])

# Perform the test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2:.4f}, p-value: {p:.4f}")

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant relationship between rocket organization and launch success.")
else:
    print("Fail to reject the null hypothesis: There is no significant relationship between rocket organization and launch success.")

The results show:

- Chi-square statistic: 27.7555,
- P-value: 0.0098

With a p-value of 0.0098, which is exceptionally smaller than the conventional significance level of 0.05, the null hypothesis is rejected, meaning that launch success is, in fact, dependent on the corporation.

The Chi-square statistic of 27.7555 points towards a very strong relationship between corporation and success. Besides that, given the low p-value, we can confirm that this relationship is statistically significant and is likely to be a real association, not being due to chance.

In conclusion, based on this chi-square test and the respective statistical value and P-value, we can conclude that organization has a meaningful influence on launch success.

We can, however, still try to figure out which company is more likely to succeed according to both raw success rate and to residual.

In [ ]:
success_rate['Confidence Interval'] = success_rate.apply(
    lambda row: (
        row['Success Rate'] - 1.96 * ((row['Success Rate'] * (1 - row['Success Rate'])) / row['Total Launches'])**0.5,
        row['Success Rate'] + 1.96 * ((row['Success Rate'] * (1 - row['Success Rate'])) / row['Total Launches'])**0.5,
    ),
    axis=1
)

contingency_table = pd.crosstab(df_balanced['Rocket Organisation'], df_balanced['Launch Success Binary'])

# Perform the Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Calculate residuals
residuals = (contingency_table - expected) / np.sqrt(expected)

# Sort residuals
residuals = residuals.sum(axis=1).sort_values(ascending=False)
print(residuals)

Before concluding anything, however, we must reason about the meaning of a high or low, positive or negative residual. The residual looks to highlight, in this case, that a launch's success is more or less likely to be due to randomness and not due to the organization itself. Besides that, we must also understand that confidence intervals are meant to evaluate how consistent the impact of a feature is (in this case, the organization).

After performing the calculation of confidence intervals for each organization, performing a chi-square test and calulating residuals for each enterprise, we understand that:  

- If we are only taking into consideration raw success rate, SpaceX is a clear favorite succeed, as it holds an almost 78% success rate. 
- However, if we are only taking into consideration both the residuals and the confidence intervals and the consistency, VKS RF comes across as the most consistent organization even while not keeping a higher success-rate - in fact, it was consistently bad! 

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(
    success_rate.index,
    success_rate['Success Rate'],
    yerr=[
        success_rate['Success Rate'] - success_rate['Confidence Interval'].apply(lambda x: x[0]),
        success_rate['Confidence Interval'].apply(lambda x: x[1]) - success_rate['Success Rate']
    ],
    fmt='o', capsize=5, color='skyblue', ecolor='darkblue'
)
plt.title('Success Rates by Corporation with 95% Confidence Intervals')
plt.xlabel('Rocket Organisation')
plt.ylabel('Success Rate')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

---

### 3rd Question: Is there any country more likely to succeed in a launch?

Let's follow a Hypothesis Testing, just as in the first question.

Hypothesis
1) Null Hypothesis (Ho): A rocket's base country has no influence on the success of a launch.
2) Alternative Hypothesis (Ha): The country where the rocket is fabricated influences positively the success of a launch.

We'll start with a simple groupby operation on the Launches data frame. We will group the data by Country, and then count the number of Total launches for each country, sum the number of Successful Launches, and provide the average number (Success Rate) of successful launches.

In [ ]:
success_rate = (
    df_balanced.groupby('Country')['Launch Success Binary']
    .agg(['count', 'sum', 'mean'])
    .rename(columns={'count': 'Total Launches', 'sum': 'Successful Launches', 'mean': 'Success Rate'})
)

success_rate = success_rate[success_rate['Total Launches'] >= 5]

success_rate = success_rate.sort_values(by=['Success Rate', 'Total Launches'], ascending=[False, False])

success_rate.head()

Let's run a visualization.

In [ ]:
plt.figure(figsize=(8, 6))
# Create an horizontal bar chart
plt.barh(success_rate.index, success_rate['Success Rate'], color='skyblue')
# Invert the y-axis
plt.gca().invert_yaxis()
plt.title('Top Rocket Organisations by Success Rate', fontsize=16)
plt.xlabel('Success Rate', fontsize=12)
plt.ylabel('Rocket Organisation', fontsize=12)
# Add a grid
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

The visualisation concludes that United States is the most successful Country within this dataset, if we consider only the average of Successful Launches per Country.

Let's dive deeper to understand the data, and see if we can find some relationship between the Rocket's Home Country and the launch success.

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_balanced['Country'], df_balanced['Launch Success Binary'])

# Perform the test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2:.4f}, p-value: {p:.4f}")

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant relationship between the Country and launch success.")
else:
    print("Fail to reject the null hypothesis: There is no significant relationship between the Country and the launch success.")

To examine the relationship between home country and launch success, we conducted a chi-square independence test (you can get more detailed information on this matter here: https://www.jmp.com/en_be/statistics-knowledge-portal/chi-square-test/chi-square-test-of-independence.html). This test evaluates the independence of a feature (launch success: 0 or 1) from another (country). The results show:

- Chi-square statistic: 9.7005,
- P-value: 0.0842

With a p-value of 0.0842, which is bigger than the conventional significance level of 0.05, the null hypothesis cannot be rejected, meaning there's no evidence that there's a relationship between the Launch Country and having a Successful Launch. 

The Chi-square statistic of 9.7005 points towards a stronger relationship between country and success, even if it's not a meaningful relationship. Taking this  p-value into consideration, we can conclude that there is insufficient evidence to conclude that the observed and expected distributions are significantly different or that the variables are dependent.

We can, however, still try to figure out which country is more likely to succeed according to both raw success rate and to residual.

In [ ]:
success_rate['Confidence Interval'] = success_rate.apply(
    lambda row: (
        row['Success Rate'] - 1.96 * ((row['Success Rate'] * (1 - row['Success Rate'])) / row['Total Launches'])**0.5,
        row['Success Rate'] + 1.96 * ((row['Success Rate'] * (1 - row['Success Rate'])) / row['Total Launches'])**0.5,
    ),
    axis=1
)

contingency_table = pd.crosstab(df_balanced['Country'], df_balanced['Launch Success Binary'])

# Perform the Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Calculate residuals
residuals = (contingency_table - expected) / np.sqrt(expected)

# Sort residuals
residuals = residuals.sum(axis=1).sort_values(ascending=False)
print(residuals)

As discussed before, the residual looks to highlight, in this case, that a launch's success is more or less likely to be due to randomness and not due to the country itself. Besides that, we must also remember that confidence intervals are meant to evaluate how consistent the impact of a feature is (in this case, the country).

After performing the calculation of confidence intervals for each country, as well as performing a chi-square test and calulating residuals for each one, we understand that:  

- If we are only taking into consideration raw success rate, United States is a clear favorite succeed, as it holds a rounded 83% success rate. 
- However, if we are only taking into consideration both the residuals and the confidence intervals and the consistency, Kazakhstan comes across as the most consistent organization even while not keeping a higher success-rate. 

In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(
    success_rate.index,
    success_rate['Success Rate'],
    yerr=[
        success_rate['Success Rate'] - success_rate['Confidence Interval'].apply(lambda x: x[0]),
        success_rate['Confidence Interval'].apply(lambda x: x[1]) - success_rate['Success Rate']
    ],
    fmt='o', capsize=5, color='skyblue', ecolor='darkblue'
)
plt.title('Success Rates by Country with 95% Confidence Intervals')
plt.xlabel('Country')
plt.ylabel('Success Rate')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


# Data Modeling

In [ ]:
df_balanced.info()

encoding

In [ ]:
# Encode categorical variables
categorical_columns = [
    'Country', 'Country_Code', 'Operator', 'Launch Site',
    'Comb Launch Site', 'Company Country', 'Ownership', 'Config',
    'Status', 'Rocket Name', 'Rocket Organisation'
]

label_encoders = {}
for col in categorical_columns:
    if col in launch_data.columns:
        label_encoders[col] = LabelEncoder()
        launch_data[f'{col}_encoded'] = label_encoders[col].fit_transform(launch_data[col])

# Scale numerical features
numerical_columns = [
    'Launch Hour', 'Launch Month', 'Launch Day', 'Launch Weekday',
    'Lat', 'Lon', 'Launch Site Lat', 'Launch Site Lon',
    'Operator Lat', 'Operator Lon', 'Rocket Price',
    'Rocket Payload to LEO', 'USD/kg to LEO',
    'USD/kg to LEO CPI Adjusted', 'Rocket Price CPI Adjusted',
    'Liftoff Thrust', 'Payload to GTO', 'Stages', 'Strap-ons',
    'Rocket Height', 'Fairing Diameter', 'Fairing Height',
    'Missions', 'Successes', 'Partial Failures', 'Failures',
    'Success Streak', 'Success Rate', 'Mass', 'Payloads'
]

scaler = StandardScaler()
numerical_columns = [col for col in numerical_columns if col in launch_data.columns]
launch_data[numerical_columns] = scaler.fit_transform(launch_data[numerical_columns].fillna(0))

correlation

In [ ]:
df_balanced.info()

In [ ]:
correlations = df_balanced[numerical_columns + ['Launch Success Binary']].corr()
correlation_with_target = correlations['Launch Success Binary'].sort_values(ascending=False)

# Display correlation results
print("Correlation with Launch Success Binary:\n", correlation_with_target)

In [ ]:
#sorted_corr = correlations.sort_values(by='Launch Success Binary', ascending=False)

# Set the figure size to ensure all values fit
plt.figure(figsize=(16, 14))  # Adjust the size as needed

# Plot the heatmap
sb.heatmap(correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

plt.show()


In [ ]:
features = df_balanced.drop(['Launch Success Binary'], axis=1)
target = df_balanced['Launch Success Binary']

nmp.unique(target, return_counts=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,target,test_size=0.1, random_state=42)
nmp.unique(y_train, return_counts=True), nmp.unique(y_test, return_counts=True)

In [ ]:
x_train_majority, y_train_majority = resample(x_train[y_train==0], y_train[y_train==0], random_state=42)
x_train_minority, y_train_minority = resample(x_train[y_train==1], y_train[y_train==1])

x_train = nmp.concatenate((x_train_majority, x_train_minority))
y_train = nmp.concatenate((y_train_majority, y_train_minority))

nmp.unique(y_train, return_counts=True), nmp.unique(y_test, return_counts=True)

In [ ]:
classification = setup(data = df_balanced, train_size=0.9, target = 'Launch Success Binary', normalize=True, transformation=True)

best = compare_models()

results = pull()
results
plot_model(best, plot='confusion_matrix', plot_kwargs={'percent' : True})